In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy

In [64]:
class Bernoulli:
    """
    The Bernoulli distribution is a discrete probability distribution of a random variable.
    The distribution has the value 1 with probability p and the value 0 with q = 1-p. 
    The Bernoulli is identical to the binomial distribution with n=1.
    One example of a Bernoulli distribution is a coin toss.
    """
    
    def __init__(self, p: float):
        assert isinstance(p, float)
        assert 0 <= p <= 1
        self.p = p
    
    def generate_random(self):
        """Return a random variable from a Bernoulli distribution"""
        random_float = random.random()
        return 1*(random_float < self.p) + 0
    
    def expected_value(self):
        """Return the expected value of a Bernoulli distribution"""
        return self.p
    
    def variance(self):
        """Return the variance of a Bernoulli distribution"""
        return self.p*(1-self.p)
    

In [147]:
class Geometric:
    """
    The geometric distribution is a distribution of n Bernoulli(p) trials required to get one success. In other words,
    there are n-1 Bernoulli failures before one Bernoulli success. The support is X = {1,2,3,...}.

    An example of a geometric distribution is the number of times a die must be thrown until a particular value occurs,
    which would be geometric(0.6)
    """
    
    def __init__(self, p: float):
        assert isinstance(p, float)
        assert 0 <= p <= 1
        self.p = p
    
    def generate_random(self):
        """Return a random variable from a Geometric distribution"""
        U = random.random()
        
        # the CDF is 1-(1-p)^floor(x)
        # the inverse of the CDF is log_(1-p) of (-U-1) = x
        # given the distribution of U-1 is the same as the distribution of U and 
        # given ln(x)/ln(y) = log_y(x) we can simplify to
        # x = ln(U)/ln(1-p) + 1

        x = np.log(U) / np.log(1-self.p)
        return int(np.floor(x)+1) 
    
    def expected_value(self):
        """Return the expected value of a Geometric distribution"""
        return 1/self.p
    
    def variance(self):
        """Return the variance of a Geometric distribution"""
        return (1-self.p) / (self.p)**2

In [207]:
class Exponential:
    """
    The exponential distribution is the distribution of the distance between events in a Poisson counting process.

    An example of an exponential distribution is inter-arrival times in a queue.

    The support (lambda), which represents the average distence between Poisson events, falls between 0 and infinity
    """
    
    def __init__(self, lambda_param: float):
        assert isinstance(lambda_param, (float, int))
        assert 0 < lambda_param
        self.lam = lambda_param
        # note that we don't want to say "lambda" alone since python uses this for defining lambda functions :)
    
    def generate_random(self):
        """Return a random variable from an exponential distribution"""
        
        # the CDF is 1-exp(-lambda * x)
        # so the inverse is X = (-1/lambda)ln(1-U) which is equivalent to (-1/lambda)ln(U)

        U = random.random()
        x = (-1 / self.lam) * np.log(U)
        return x
    
    def expected_value(self):
        """Return the expected value of an exponential distribution"""
        return 1/self.lam
    
    def variance(self):
        """Return the variance of an exponential distribution"""
        return 1/(self.lam**2)

In [229]:
class Uniform:
    """
    The continuous uniform distribution is bounded by the parameters [a,b]. As this distribution is uniform,
    the PDF is 1/(b-a) for a<= x <=b and 0 elsewhere; in other words, the probability distribution is equal for anywhere
    in the range between a and b
    """
    
    def __init__(self, a: float, b: float):
        assert isinstance(a, (float, int))
        assert isinstance(b, (float, int))
        assert 0 < a < b
        self.a = a
        self.b = b
    
    def generate_random(self):
        """Return a random variable from a uniform distribution"""
        
        U = random.random()
        x = self.a + (self.b-self.a)*U
        return x
    
    def expected_value(self):
        """Return the expected value of a uniform distribution"""
        return (self.a + self.b) / 2
    
    def variance(self):
        """Return the variance of a uniform distribution"""
        return ((self.b - self.a) ** 2) / 12

In [ ]:
class Weibull:
    """
    The Weibull distribution primarily models time between events, particularly time to failure.

    The Weibull distribution takes two parameters, alpha and beta:
    - alpha is the "shape" parameter
    - beta is the "scale" parameter

    For alpha < 1, event rate (e.g. failures) decreases over time, e.g. modeling defects that fail early
    For alpha = 1, event rate is static over time
    For alpha > 1, event rate increases over time, e.g. modeling parts wearing out
    """
    
    def __init__(self, alpha: float, beta: float):
        assert isinstance(alpha, (float, int))
        assert isinstance(beta, (float, int))
        assert 0 < alpha
        assert 0 < beta
        self.alpha = alpha
        self.beta = beta
    
    def generate_random(self):
        """Return a random variable from a uniform distribution"""
        
        U = random.random()
        x = self.alpha * -np.log(U)**(1/self.beta)
        return x
    
    def expected_value(self):
        """Return the expected value of a uniform distribution"""
        return (self.a + self.b) / 2
    
    def variance(self):
        """Return the variance of a uniform distribution"""
        return ((self.b - self.a) ** 2) / 12